# Scrapping

Responsable : *Dylan Rousseau*

### **Objet :**  
....................................................
<br/>    

---  
Historique des modifications : 

21/03/2022 : *Dylan ROUSSEAU* : *Création du script*<br/> 

---

# Imports bibliothèques

In [1]:
import math
import numpy
import pandas as pd
import ipywidgets as widgets
import IPython.display as ipd

from bs4 import BeautifulSoup
from IPython import display as audioD
from requests_html import HTMLSession
from IPython.display import display, clear_output, HTML, Markdown 

C:\Users\droussea\Anaconda3\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\droussea\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.NOIJJG62EMASZI6NYURL6JBKM4EVBGM7.gfortran-win_amd64.dll
C:\Users\droussea\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
  stacklevel=1)


# Variables

In [2]:
session   = HTMLSession()
dataFrame = widgets.Output()
audio     = widgets.Output()

# Imports des datasources

In [3]:
dfDicoBuch = pd.read_csv('../data/dfDicoBuch.csv')
dfDicoBuch = dfDicoBuch[["motVedette","nbSyll","pron","pronSampa","ipa"]]
dfDicoBuch = dfDicoBuch.replace(numpy.nan, '', regex=True)

# Widgets

In [4]:
filtreHWD = widgets.Text(value="",description='filtre hwd',layout={'width': '200px'} )
filtreIPA = widgets.Text(value="",description='filtre ipa',layout={'width': '200px'} )
occMax    = widgets.IntSlider(min=0,max=10,step=1,description='Occ.Max:',value = 5)
bouton    = widgets.Button(description='Démarrer', icon = "check")

# Fonctions génériques

In [5]:
def search_ldoceonline_mp3(dfEtudie):
    
    listLinknUK = []
    listLinknUS = []
    
    for word in dfEtudie["motVedette"] :

        # On créé le lien URL
        urlTarget = "https://www.ldoceonline.com/dictionary/" + word
        response  = session.get(urlTarget)
        if response.status_code == 200:
            html_doc  = response.content
            structure = BeautifulSoup(response.text,'html.parser')
            checkBody = structure.find('body')
            checkDico = checkBody.findAll('div',{"class":"dictionary"})
            for x in checkDico:
                mpsUK = x.findAll('span',{"class":"speaker brefile fas fa-volume-up hideOnAmp"})
            for y in mpsUK:
                linkUK = y['data-src-mp3']
            checkLinkUK = linkUK.replace(linkUK[-15:],"")
            listLinknUK.append(checkLinkUK)

            for x in checkDico:
                mpsUS = x.findAll('span',{"class":"speaker amefile fas fa-volume-up hideOnAmp"})
            for y in mpsUS:
                linkUS = y['data-src-mp3']
            checkLinkUS = linkUS.replace(linkUS[-15:],"")
            listLinknUS.append(checkLinkUS)
        else:
            print('NUL')
    dfEtudie["linkUK"] = pd.Series(listLinknUK).values
    dfEtudie["linkUS"] = pd.Series(listLinknUS).values
        
        
    return dfEtudie

In [6]:
def creaAudio(dfEtudie):
    clear_output()
    listeWid = widgets.Output()
    lU = widgets.Output()

    for numLigne in range(len(dfEtudie)):

        labelUK = widgets.Label("Brit")
        itemsUK = audioD.Audio(url = dfEtudie["linkUK"][numLigne])
        labelUS = widgets.Label("Amer")
        itemsUS = audioD.Audio(url = dfEtudie["linkUS"][numLigne])
        audioUK = widgets.Output()
        audioUK.append_display_data(itemsUK)
        audioUS = widgets.Output()
        audioUS.append_display_data(itemsUS)

        lU = widgets.HBox([labelUK,audioUK,labelUS,audioUS])

        # Les mots

        labelMot = widgets.Label(dfEtudie["motVedette"][numLigne])
        labelPron = widgets.Label(str(dfEtudie["pron"][numLigne]))
        label = widgets.HBox([labelMot, labelPron])


        listeWid.append_display_data(widgets.VBox([label,lU]))
                        
    return listeWid

In [7]:
def on_button_clicked(_):
    nbLigne = occMax.value
    HWD = filtreHWD.value 
    IPA = filtreIPA.value
    dfEtudie = ""

    with dataFrame:
        clear_output()
        # Filtre en fontion des variables précédentes 
        if (HWD != ""):
            dfEtudie = dfDicoBuch[(dfDicoBuch.motVedette.str.match(HWD))]
            if(IPA != ""):
                dfEtudie = dfEtudie[(dfEtudie.pron.str.match(IPA))]
        elif (IPA != ""):
            dfEtudie = dfDicoBuch[(dfDicoBuch.pron.str.match(IPA))]

        if(len(dfEtudie) > nbLigne):
            dfEtudie = dfEtudie.sample(n=nbLigne).copy()
        dfEtudie = dfEtudie.reset_index()    
        # On affiche
        display(dfEtudie)
        
    with audio :
        clear_output()
        dfEtudie = search_ldoceonline_mp3(dfEtudie)  
        
        listeAudio = creaAudio(dfEtudie) 
        display(listeAudio)
        
            
bouton.on_click(on_button_clicked)

# Start layout 

In [8]:
ligne1 = widgets.HBox([filtreHWD, filtreIPA, occMax, bouton])
ligne2 = widgets.HBox([dataFrame])
ligne3 = widgets.HBox([audio])

result = widgets.VBox([ligne1, ligne2, ligne3])

# Lancement de l'appli

In [9]:
result